# ECML/PKDD 15: Taxi Trajectory Prediction (I)

此為在Kaggle比賽中找到的一個以前的比賽，是用計程車前面經過的地點(經緯度)來預測他的目的地(經緯度)

我覺得此題目重要並有趣的原因是在於，若預測準確，可以提早知道乘客下車的地點，並能通知司機知道他下一個能服務的乘客距離此地點近不近，他可以大大提升計程車公司的效率。

此比賽在kaggle上的評分標準是用Mean Haversine Distance，它根據緯度和經度測量球體上兩點之間的距離(這樣才不會因為地球是球體而有偏誤)，而我目前用神經網路得到的分數是3.54468，雖然名次不是很漂亮(名次在總體的2/3左右)，但我覺得是模型的限制，若是之後學習到更適合處理time series資料的模型，相信可以得到更好的分數

p.s.為了想練習tensorflow此報告我使用tensorflow實做

![分數](mid2000_200_20_354468.PNG)

In [1]:
import tensorflow as tf
import numpy as np
import os
import pandas as pd

In [2]:
#定義layer
def add_layer(inputs, in_size, out_size, activation_function=None):
    Weights = tf.Variable(tf.random_normal([in_size, out_size]))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    
    #選擇激活函數
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    return outputs

In [5]:
x_data = pd.read_csv("traindata.csv")
y_data = pd.read_csv("label.csv")

In [6]:
#將Nan轉為0
x_data = x_data.fillna(0)
y_data = y_data.fillna(0)

In [7]:
#將不必要的欄位切掉(每次讀只能跑一次)
x_data=x_data.iloc[:,1:]
y_data=y_data.iloc[:,1:]

In [8]:
x_data = x_data.iloc[0:10000]
y_data = y_data.iloc[0:10000]

In [9]:
len(x_data)

10000

In [10]:
len(x_data.iloc[0])

5030

In [11]:
xs = tf.placeholder(tf.float32, [None, 5030])
ys = tf.placeholder(tf.float32, [None, 2])

In [12]:
l1 = add_layer(xs, 5030, 2000, activation_function = tf.nn.tanh)
l2 = add_layer(l1, 2000, 200, activation_function = tf.nn.tanh)
l3 = add_layer(l2, 200, 20, activation_function = tf.nn.tanh)
outputlayer = add_layer(l3, 20, 2, activation_function = None)
loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - outputlayer)))
# adam是改良的gd ,learning rate會越來越小
train_step = tf.train.AdamOptimizer(0.03).minimize(loss)

In [13]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
batch = 40

In [14]:
for i in range(250):
    print(i)
    sess.run(train_step, feed_dict={xs: x_data.iloc[(batch*i):(batch*(i+1)),:], ys: y_data.iloc[(batch*i):(batch*(i+1)),:]})
    print(sess.run(loss, feed_dict={xs: x_data.iloc[(batch*i):(batch*(i+1)),:], ys: y_data.iloc[(batch*i):(batch*(i+1)),:]}))
    print(sess.run(outputlayer, feed_dict={xs: x_data.iloc[batch*i:batch*(i+1),:], ys: y_data.iloc[batch*i:batch*(i+1),:]}))

0
38169.695
[[-6.63271964e-01  1.28269320e+01]
 [ 3.02976823e+00  1.50880728e+01]
 [ 5.15637350e+00  1.56364985e+01]
 [ 7.69720030e+00  1.12390871e+01]
 [ 4.98141587e-01  1.13589020e+01]
 [-3.17311454e+00  1.03342533e+01]
 [ 1.73270464e+00  1.15152426e+01]
 [-1.10145414e+00  1.37578621e+01]
 [-5.94117582e-01  1.28366785e+01]
 [ 2.13565803e+00  1.38602095e+01]
 [ 8.54679883e-01  1.52788811e+01]
 [ 2.39189315e+00  1.19525490e+01]
 [ 1.67278898e+00  1.15004435e+01]
 [-7.26069093e-01  9.76208210e+00]
 [ 1.58105642e-02  1.59996662e+01]
 [ 3.42044282e+00  1.50763407e+01]
 [-6.15553081e-01  1.28357954e+01]
 [ 5.41538715e+00  1.38014650e+01]
 [ 6.29165173e+00  1.00692282e+01]
 [ 3.93590617e+00  1.17471476e+01]
 [ 5.15759039e+00  1.56374998e+01]
 [ 4.23564911e+00  7.94264317e+00]
 [-8.97597730e-01  1.31243925e+01]
 [ 2.59899473e+00  1.10683622e+01]
 [ 6.08633471e+00  1.16355391e+01]
 [ 1.31894946e+00  1.46866322e+01]
 [ 7.75789499e+00  1.12361746e+01]
 [ 5.15103197e+00  1.38122015e+01]
 [ 1.110

20053.51
[[-9.30474   19.097576 ]
 [-6.0072904 19.391012 ]
 [-6.0213985 19.389757 ]
 [-9.302525  19.09777  ]
 [-6.007252  19.391016 ]
 [-6.00746   19.390997 ]
 [-6.007252  19.391016 ]
 [-8.182537  19.19744  ]
 [-9.304734  19.097574 ]
 [-6.007252  19.391016 ]
 [-6.007251  19.391016 ]
 [-6.2068257 19.373255 ]
 [-6.007252  19.391016 ]
 [-9.304755  19.097574 ]
 [-6.007252  19.391016 ]
 [-9.274457  19.10027  ]
 [-6.0072546 19.391016 ]
 [-9.285285  19.102198 ]
 [-6.206818  19.373257 ]
 [-6.0102654 19.390749 ]
 [-5.5186157 19.996836 ]
 [-9.30468   19.097591 ]
 [-9.301822  19.097834 ]
 [-7.2086883 19.284101 ]
 [-9.303887  19.09765  ]
 [-9.1914835 19.107653 ]
 [ 6.654299   5.521814 ]
 [-6.007252  19.391016 ]
 [-6.0082917 19.390924 ]
 [-9.061845  19.119188 ]
 [-6.007252  19.391016 ]
 [-6.0077767 19.39097  ]
 [-6.0073957 19.391005 ]
 [-6.0085216 19.390903 ]
 [-9.015544  19.123308 ]
 [-6.007252  19.391016 ]
 [-6.032545  19.388765 ]
 [-9.280626  19.09972  ]
 [-6.2069488 19.373245 ]
 [-6.007252  19.

11611.372
[[-5.159261  24.485256 ]
 [-5.15926   24.485256 ]
 [-5.1592607 24.485256 ]
 [-5.15926   24.485256 ]
 [-5.159262  24.485256 ]
 [-5.15926   24.485256 ]
 [-5.1592603 24.485256 ]
 [-5.15926   24.485256 ]
 [-5.15926   24.485256 ]
 [-5.1592607 24.485256 ]
 [-5.15926   24.485256 ]
 [-5.160199  24.485054 ]
 [-5.1601973 24.485054 ]
 [-5.15926   24.485256 ]
 [-5.15926   24.485256 ]
 [-5.1592607 24.485256 ]
 [-5.15926   24.485256 ]
 [-5.15926   24.485256 ]
 [-5.15926   24.485256 ]
 [-5.15926   24.485256 ]
 [-5.15926   24.485256 ]
 [-5.15926   24.485256 ]
 [-5.1592603 24.485256 ]
 [-5.1592607 24.485256 ]
 [-5.15926   24.485256 ]
 [-5.15926   24.485256 ]
 [-5.1592593 24.485256 ]
 [-5.15926   24.485256 ]
 [-5.15926   24.485256 ]
 [-5.15926   24.485256 ]
 [-5.1592607 24.485256 ]
 [-5.15926   24.485256 ]
 [-5.159261  24.485256 ]
 [-5.1593094 24.485249 ]
 [-5.15926   24.485256 ]
 [-5.15926   24.485256 ]
 [-5.15926   24.485256 ]
 [-5.1612153 24.484837 ]
 [-5.159261  24.485256 ]
 [-5.1592607 24

 [-9.093767 29.312527]]
25
6357.1147
[[-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [ 6.3355093 10.530773 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  29.825756 ]
 [-9.386678  

2092.654
[[-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.02665  ]
 [-9.8486595 34.

...

120.62472
[[-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [31.014051  -4.2514887]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41.156418 ]
 [-8.622173  41

103.87426
[[-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [31.852953 -4.177914]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]
 [-8.619998 41.15745 ]]
219
0.035889514
[[-8.621219 41.15741 ]
 [-8.621219 41.15741 ]
 [-8.62

1.1346935
[[-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]
 [-8.616311 41.157192]]
229
93.776985
[[-8.615124  41.157272 ]
 [-8.615124  41.157272 ]
 [-8.

0.10323319
[[-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]
 [-8.637632 41.16228 ]]
239
0.09573285
[[-8.639136 41.16294 ]
 [-8.639136 41.16294 ]
 [-8.63

0.033553746
[[-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]
 [-8.620984 41.166256]]
249
0.04890769
[[-8.619431 41.16621 ]
 [-8.619431 41.16621 ]
 [-8.6

In [17]:
#用test.csv做預測後上傳檔案至kaggle
dft = pd.read_csv("test.csv")
dft = dft["POLYLINE"].str.replace("[","")
dft = dft.str.replace("],",",")
dft = dft.str.replace("]","")
dft = dft.str.split(",",-1,True)
dft.fillna(value=0, inplace=True)
dft = dft.astype(float)

In [18]:
dft

,0,1,2,3,4,5,6,7,8,9,...,1214,1215,1216,1217,1218,1219,1220,1221,1222,1223
0,-8.585676,41.148522,-8.585712,41.148639,-8.585685,41.148855,-8.585730,41.148927,-8.585982,41.148963,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-8.610876,41.145570,-8.610858,41.145579,-8.610903,41.145768,-8.610444,41.146191,-8.609445,41.146758,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-8.585739,41.148558,-8.585730,41.148828,-8.585721,41.148972,-8.586288,41.149017,-8.586117,41.148567,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-8.613963,41.141169,-8.614125,41.141124,-8.615088,41.140926,-8.615277,41.140818,-8.615259,41.140809,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-8.619903,41.148036,-8.619894,41.148036,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,-8.630613,41.178249,-8.630613,41.178249,-8.630739,41.178231,-8.631513,41.178141,-8.631306,41.178114,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,-8.585622,41.148918,-8.585640,41.148900,-8.585919,41.148945,-8.586369,41.148927,-8.586126,41.148558,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,-8.582922,41.181057,-8.582004,41.181813,-8.581077,41.182965,-8.580114,41.184018,-8.579214,41.184774,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,-8.606529,41.144670,-8.606673,41.144724,-8.606799,41.144706,-8.606790,41.144679,-8.606709,41.144724,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,-8.585658,41.148576,-8.585703,41.148603,-8.585730,41.148603,-8.585739,41.148603,-8.585730,41.148612,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
dfnew = pd.DataFrame(np.zeros((len(dft),(len(x_data.iloc[0]) - len(dft.iloc[0]))))) 
dft = pd.concat([dft, dfnew],axis = 1)

In [20]:
output = sess.run(outputlayer, feed_dict = {xs : dft})

In [21]:
#output.csv需要打開更改row跟col的名稱才能上傳kaggle
output  = pd.DataFrame(output)
output.to_csv("outputmidtest2.csv")